In [1]:
import rospy
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from tf.transformations import euler_from_quaternion
import numpy as np
import math
from delaunay2D import Delaunay2D

# 机器人类，包含机器人的位置信息，以及机器人的运动控制
class robot:
    def __init__(self, name, x, y, theta):
        self.name = name
        self.x = x
        self.y = y
        self.theta = theta
        # 增量更新，因此记录上一次odom信息
        self.last_x = x
        self.last_y = y
        self.last_theta = theta
        self.pub = rospy.Publisher('/%s/cmd_vel' % name, Twist, queue_size=10)

    # 使用里程计信息更新机器人位姿
    def update_with_odom(self, data):
        # 计算位置增量
        delta_x = data.pose.pose.position.x - self.last_x
        delta_y = data.pose.pose.position.y - self.last_y
        # 计算角度增量
        theta_now = euler_from_quaternion([data.pose.pose.orientation.x, data.pose.pose.orientation.y,
                                             data.pose.pose.orientation.z, data.pose.pose.orientation.w])[2]
        delta_theta = theta_now - self.last_theta
        # 更新位姿
        self.x += delta_x
        self.y += delta_y
        self.theta += delta_theta
        # 更新上一时刻位姿
        self.last_x = data.pose.pose.position.x
        self.last_y = data.pose.pose.position.y
        self.last_theta = theta_now

    # 使用后端信息更新机器人位姿
    def update_with_optimization(self, x, y, theta):
        self.x = x
        self.y = y
        self.theta = theta

    # 发布控制命令
    def move(self, vx, vy, vtheta):
        move_cmd = Twist()
        move_cmd.linear.x = vx
        move_cmd.linear.y = vy
        move_cmd.angular.z = vtheta
        self.pub.publish(move_cmd)
    

# 工具类，订阅真实位姿，计算虚拟位姿，以及相互测距结果（矩阵形式，1代表可以相互测距,0代表机器人间没有信号）
class msg_generator:
    pass

# 工具类，对机器人集群进行全局优化
class global_optimization:
    pass

# 几何工具包
class geometry_tool:
    # 计算三角形重心
    def triangle_center(self, p1, p2, p3):
        return (p1+p2+p3)/3
    
    # 计算delaunay三角形
    def delaunay(self, point_list):
        dt = Delaunay2D()
        # Insert all seeds one by one
        for point in point_list:
            dt.addPoint(point)
        return dt.exportTriangles()


# 机器人集群类，包含多个机器人，以及机器人间的相互位置信息
class robot_group:
    def __init__(self, robot_list):
        self.robot_list = robot_list
        self.robot_num = len(robot_list)
        # 机器人下一次的速度控制量
        self.robot_vx = np.zeros(self.robot_num)
        self.robot_vy = np.zeros(self.robot_num)
        self.robot_vtheta = np.zeros(self.robot_num)
        # 二维数组，用于记录相互测距结果
        self.robot_dist = np.zeros((self.robot_num, self.robot_num))
        # 参数
        self.dist_min = 0.5
        self.dist_max = 1.5

    # 距离墙体的距离系数，距离越近，系数越大
    def wall_force(self, dist):
        return 1/(dist+0.1)

    # 机器人相互距离系数，小于dist_min时，越近越排斥，大于dist_max时，越远越吸引，中间时，距离越近，系数越大
    def robot_force(self, dist, dist_min, dist_max):
        if dist < dist_min:
            return 1/(dist+0.1)
        elif dist > dist_max:
            return -dist**2
        else:
            return 1/(dist+0.1)
    

    # 用odom信息更新机器人位置
    def update_robots_with_odom(self):
        for robot in self.robot_list:
            data = rospy.wait_for_message('/%s/odom' % robot.name, Odometry)
            robot.update_with_odom(data)

    # 用全局测距信息更新机器人位置
    def update_robots_with_optimization(self):
        # 更新全局位置，并加入噪声
        for robot in self.robot_list:
            data = rospy.wait_for_message('/%s/base_pose_ground_truth' % robot.name, Odometry)
            x = data.pose.pose.position.x+np.random.normal(0, 0.1)
            y = data.pose.pose.position.y+np.random.normal(0, 0.1)
            theta = euler_from_quaternion([data.pose.pose.orientation.x, data.pose.pose.orientation.y,
                                                    data.pose.pose.orientation.z, data.pose.pose.orientation.w])[2]
            robot.update_with_optimization(x,y,theta)

    # 更新全局相互测距结果
    def update_robot_dist(self):
        for i in range(self.robot_num):
            for j in range(self.robot_num):
                if i != j:
                    distance = math.sqrt((self.robot_list[i].x-self.robot_list[j].x)**2+(self.robot_list[i].y-self.robot_list[j].y)**2)
                    if distance < 1.5 and distance > 0:
                        self.robot_dist[i][j] = distance
                    else:
                        self.robot_dist[i][j] = -1
        

    # 控制所有机器人运动
    def controll_all_robots(self):
        # 按照机器人顺序加入机器人坐标，并生成delaunay三角形
        point_list = []
        for robot in self.robot_list:
            point_list.append([robot.x, robot.y])
        dt = geometry_tool().delaunay(point_list)
        # 计算三角形重心以重心到顶点的向量为基础，加入距离墙体的影响，加入距离其他机器人的影响
        for triangle in dt:
            center = geometry_tool().triangle_center(np.array(point_list[triangle[0]]), np.array(point_list[triangle[1]]), np.array(point_list[triangle[2]]))
            for i in range(3):
                # 计算重心到顶点的向量
                vector = np.array(point_list[triangle[i]])-center
                # 根据距离计算系数
                robot_force =  self.robot_force(math.sqrt(vector[0]**2+vector[1]**2), self.dist_min, self.dist_max)
                # 计算速度(先归一化，再乘以系数)
                v = vector/math.sqrt(vector[0]**2+vector[1]**2)*robot_force
                self.robot_vx[triangle[i]] += v[0]
                self.robot_vy[triangle[i]] += v[1]
                self.robot_vtheta[triangle[i]] += 0
            
        # 根据合成速度发布控制命令
        for i in range(self.robot_num):
            self.robot_list[i].move(self.robot_vx[i], self.robot_vy[i], self.robot_vtheta[i])

In [2]:
rospy.init_node('robot_network_controller_ipy', anonymous=True)
robot0=robot('robot_0', 0, 0, 0)
robot1=robot('robot_1', 0, 0, 0)
robot2=robot('robot_2', 0, 0, 0)
robot3=robot('robot_3', 0, 0, 0)

Unable to register with master node [http://192.168.2.11:11311]: master may not be running yet. Will keep trying.


In [27]:
robot0.move(1,1,0)